# Binary classification risk control - Theoretical tests to validate implementation

# 1. The case of a random classifier

## 1.1. Protocol description
We test the theoretical guarantees of risk control in binary classification by using a random classifier and synthetic data. The aim is to evaluate the effectiveness of the BinaryClassificationController in maintaining a predefined risk level under different conditions.

Each test case looks at a combination of parameters, for which we repeat the experiment `n_repeat` times. The model is the same for all experiments (basically a random classifier), but the data is different each time.

Each experiment consists of the following:
 - We calibrate a BinaryClassificationController. It gives us the list of lambda values that control the risk according to LTT.
 - Because we know that the model is random, we know the theoretical risk associated with each lambda value. So we are able to check if the lambda values given by LTT actually control the risk. If not, we count 1 "error". Note that *each* lambda value should control the risk, not just one of them.

After n_repeat experiments, we compute the proportion of errors, that should be less than delta (1 - confidence_level).

## 1.2. Results
The risk is controlled in all the test cases. Overall, LTT seems very conservative (to achieve a high percentage of errors, we need to lower the confidence level significantly (0.01) and use only one threshold to avoid the Bonferroni effect). But this is likely due to the model being random, and thus having a lot of variance. It would be interesting to see how this evolves with a better model.

In [3]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from sklearn.datasets import make_classification
from sklearn.dummy import check_random_state
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
from mapie.risk_control import BinaryClassificationController, precision, accuracy, recall
from itertools import product
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Using sklearn.dummy.DummyClassifier would be cleaner
class RandomClassifier:
    def __init__(self, seed=42, threshold=0.5):
        self.seed = seed
        self.threshold = threshold

    def _get_prob(self, x):
        local_seed = hash((x, self.seed)) % (2**32)
        rng = np.random.RandomState(local_seed)
        return np.round(rng.rand(), 2)

    def predict_proba(self, X):
        probs = np.array([self._get_prob(x) for x in X])
        return np.vstack([1 - probs, probs]).T

    def predict(self, X):
        probs = self.predict_proba(X)[:, 1]
        return (probs >= self.threshold).astype(int)


def precision_random_classifier(threshold: float) -> float:
    """
    Theoretical precision of RandomClassifier on a balanced dataset.
    - RandomClassifier assigns labels randomly with a uniform probability, independent of the true label.
    - Therefore, precision is always 0.5 regardless of the threshold.
    """
    return 0.5


def accuracy_random_classifier(threshold: float) -> float:
    """
    Theoretical accuracy of RandomClassifier on a balanced dataset.
    - RandomClassifier assigns labels randomly with a uniform probability, independent of the true label.
    - Therefore, accuracy is always 0.5 regardless of the threshold.
    """
    return 0.5


def recall_random_classifier(threshold: float) -> float:
    """
    Theoretical recall (sensitivity) of RandomClassifier on a balanced dataset.
    - RandomClassifier assigns labels randomly with a uniform probability, independent of the true label.
    - Therefore, recall = 1 - threshold.
    """
    return 1.0 - threshold

In [ ]:
N = 2000  # size of the calibration set
risk = [
    {"name": "precision", "risk": precision},
    {"name": "recall", "risk": recall},
    {"name": "accuracy", "risk": accuracy},
]
predict_params = [np.linspace(0, 0.99, 100), np.empty(1)]
target_level = [0.1, 0.9]
confidence_level = [0.8, 0.2]

n_repeats = 100
invalid_experiment = False

for combination in product(risk, predict_params, target_level, confidence_level):
    risk, predict_params, target_level, confidence_level = combination
    if len(predict_params) == 1:
        predict_params = np.array([np.random.choice(np.linspace(0, 0.9, 10))]) # random threshold
    alpha = float(Decimal("1") - Decimal(str(target_level))) # to avoid floating point issues
    delta = float(Decimal("1") - Decimal(str(confidence_level))) # to avoid floating point issues

    clf = RandomClassifier()
    nb_errors = 0  # number of iterations where the risk is not controlled (i.e., not all the valid thresholds found by LTT are actually valid)
    total_nb_valid_params = 0

    for _ in range(n_repeats):

        X_calibrate, y_calibrate = make_classification(
            n_samples=N,
            n_features=1,
            n_informative=1,
            n_redundant=0,
            n_repeated=0,
            n_classes=2,
            n_clusters_per_class=1,
            weights=[0.5, 0.5],
            flip_y=0,
            random_state=None
        )
        X_calibrate = X_calibrate.squeeze()

        controller = BinaryClassificationController(
            predict_function=clf.predict_proba,
            risk=risk["risk"],
            target_level=target_level,
            confidence_level=confidence_level,
        )
        controller._predict_params = predict_params
        controller.calibrate(X_calibrate, y_calibrate)
        valid_parameters = controller.valid_predict_params
        total_nb_valid_params += len(valid_parameters)

        # In the following, we check that all the valid thresholds found by LTT actually control the risk.
        # Instead of sampling a large test set, we use the fact that we know the theoretical risk of a random classifier.
        # The calculations here are valid only for a balanced data generator.
        for lambda_ in valid_parameters:
            if risk["risk"] == precision:
                theoretical_metric = precision_random_classifier(lambda_)
            elif risk["risk"] == recall:
                theoretical_metric = recall_random_classifier(lambda_)
            elif risk["risk"] == accuracy:
                theoretical_metric = accuracy_random_classifier(lambda_)

            if risk["risk"].higher_is_better:
                if theoretical_metric <= target_level:
                    nb_errors += 1
                    break
            else:
                if theoretical_metric > target_level:
                    nb_errors += 1
                    break

    print(f"\n{N=}, {risk['name']=}, {len(predict_params)=}, {target_level=}, {confidence_level=}")

    print(f"Proportion of times the risk is not controlled: {nb_errors/n_repeats}")
    print(f"Delta: {delta}")
    print(f"Mean number of valid thresholds found per iteration: {int(np.round(total_nb_valid_params/n_repeats))}")

    if nb_errors/n_repeats <= delta:
        print("Valid experiment")
    else:
        print("Invalid experiment")
        invalid_experiment = True

print("\n\n\n")
if invalid_experiment:
    print("Some experiments failed.")
else:
    print("All good!")

In [5]:
assert not invalid_experiment

# 2. The case of a logistic classifier

## 2.1. Protocol description
We use the same protocol as described above, with the difference that we employ a logistic classifier and synthetic data generated from a logistic data generator.

Another difference is that, with this model, we do not know the theoretical risk associated with each value of lambda.
Therefore, we generate a sample of test data to estimate the risk for each lambda value.

## 2.2. Results
The risk is controlled in all test cases. Overall, LTT appears to be very conservative. It would be interesting to observe how this behavior evolves with a more accurate or complex model.

In [ ]:
# Define a simple logistic classifier
class LogisticClassifier:
    """Deterministic sigmoid-based binary classifier."""

    def __init__(self, scale=2.0, threshold=0.5):
        self.scale = scale
        self.threshold = threshold
    
    def sigmoid(self, x):
        """Sigmoid function."""
        return 1 / (1 + np.exp(-self.scale * x))
    
    def _get_prob(self, x):
        """Probability of class 1 for input x."""
        return self.sigmoid(x)

    def predict_proba(self, X):
        """Return probabilities [p(y=0), p(y=1)] for each sample in X."""
        probs = np.array([self._get_prob(x) for x in X])
        return np.vstack([1 - probs, probs]).T

    def predict(self, X):
        """Return predicted class labels based on threshold."""
        probs = self.predict_proba(X)[:, 1]
        return (probs >= self.threshold).astype(int)

In [ ]:
# Function to generate logistic data
def make_logistic_data(n_samples=200, scale=2.0, random_state=None):
    rng = check_random_state(random_state)
    X = rng.uniform(-3, 3, size=n_samples)
    probs = LogisticClassifier(scale=scale).sigmoid(X)
    y = rng.binomial(1, probs)
    return X, y

In [2]:
N = 100
risk = [
    {"name": "precision", "risk": precision},
    {"name": "recall", "risk": recall},
    {"name": "accuracy", "risk": accuracy},
]
predict_params =  [np.linspace(0, 0.99, 100), np.empty(1)]
target_level = [0.1, 0.9]
confidence_level = [0.8, 0.2]

n_repeats = 100
invalid_experiment = False

for combination in product(risk, predict_params, target_level, confidence_level):
    risk, predict_params, target_level, confidence_level = combination
    if len(predict_params) == 1:
        predict_params = np.array([np.random.choice(np.linspace(0, 0.9, 10))]) # random threshold
    alpha = float(Decimal("1") - Decimal(str(target_level))) # to avoid floating point issues
    delta = float(Decimal("1") - Decimal(str(confidence_level))) # to avoid floating point issues

    clf = LogisticClassifier(scale=2.0, threshold=0.5)
    nb_errors = 0  # number of iterations where the risk is not controlled (i.e., not all the valid thresholds found by LTT are actually valid)
    total_nb_valid_params = 0

    for _ in range(n_repeats):

        X_calibrate, y_calibrate = make_logistic_data(n_samples=N, scale=2.0, random_state=None)

        controller = BinaryClassificationController(
            predict_function=clf.predict_proba,
            risk=risk["risk"],
            target_level=target_level,
            confidence_level=confidence_level,
        )
        controller._predict_params = predict_params
        controller = controller.calibrate(X_calibrate, y_calibrate)
        valid_parameters = controller.valid_predict_params
        total_nb_valid_params += len(valid_parameters)

        # In the following, we check that all the valid thresholds found by LTT actually control the risk.
        # We sample a large test set and estimate the risk for each valid_parameters using the logistic classifier.
        X_test, y_test = make_logistic_data(n_samples=N, scale=2.0, random_state=None)
        probs = clf.predict_proba(X_test)[:, 1]
        
        # If no valid parameters found, risk is not controlled
        if len(valid_parameters) >= 1:
            for lambda_ in valid_parameters:
                y_pred = (probs >= lambda_).astype(int)

                if risk["risk"] == precision:
                    empirical_metric = precision_score(y_test, y_pred, zero_division=0)
                elif risk["risk"] == recall:
                    empirical_metric = recall_score(y_test, y_pred, zero_division=0)
                elif risk["risk"] == accuracy:
                    empirical_metric = accuracy_score(y_test, y_pred)

                # Check if the risk control fails
                if risk["risk"].higher_is_better:
                    if empirical_metric <= target_level:
                        nb_errors += 1
                        break 
                else:
                    if empirical_metric > target_level:
                        nb_errors += 1
                        break

    print(f"\n{N=}, {risk['name']=}, {len(predict_params)=}, {target_level=}, {confidence_level=}")

    print(f"Proportion of times the risk is not controlled: {nb_errors/n_repeats}")
    print(f"Delta: {delta}")
    print(f"Mean number of valid thresholds found per iteration: {int(np.round(total_nb_valid_params/n_repeats))}")

    if nb_errors/n_repeats <= delta:
        print("Valid experiment")
    else:
        print("Invalid experiment")
        invalid_experiment = True

print("\n\n\n")
if invalid_experiment:
    print("Some experiments failed.")
else:
    print("All good!")

NameError: name 'precision' is not defined

In [9]:
assert not invalid_experiment